In [1]:
 #accessing netcdf files from Hector's folders
import os, glob, sys
sys.path.append("//nobackup//amondal//Python//Hector_Python_Scripts")
sys.path.append("//nobackup/amondal//Python//mit_geos_analysis")
#from mds_store import openmdsdataset
#playing with netcdf - xarray
import numpy as np
import xarray as xr
import dask.array as da
import matplotlib.pyplot as plt
# How about visualization?
%matplotlib inline
import netCDF4
from netCDF4 import Dataset
#from xmitgcm import open_mdsdataset
from GetNCDataSet import *
from MITllc2160Depth import *
sys.path.append("//nobackup/amondal//Python//xmitgcm//xmitgcm")
from xmitgcm.mds_store import open_mdsdataset
import xrft
import xgcm
from spectral_analysis_code import *
from GetNCDataSet import getGEOSNCDataSet

from dask.distributed import Client
client = Client(memory_limit='100GB',n_workers = 40, threads_per_worker=1)

So for calculating Richardson number, you need a few things.
1. Definitely U, V, T, S (U,V for shear, T, S, for density). 
2. Take appropriate derivatives

In [ ]:
import numpy as np
import gsw
import dens # hector can give you this module if you need it
import glob
import scipy.io as io


## load some grid parameters
YC = np.memmap(prnt+'YC_'+str(NX)+'x'+str(NY), dtype = '>f4',shape=(NY,NX), mode='r').astype(float)[1177:-2,1199:2001]
DYF = np.memmap(prnt+'DYF_'+str(NX)+'x'+str(NY), dtype = '>f4',shape=(NY,NX), mode='r').astype(float)[1177:-2,1199:2001]
DXF = np.memmap(prnt+'DXF_'+str(NX)+'x'+str(NY), dtype = '>f4',shape=(NY,NX), mode='r').astype(float)[1177:-2,1199:2001]
RAU = np.memmap(prnt+'RAW_'+str(NX)+'x'+str(NY), dtype = '>f4',shape=(NY,NX), mode='r').astype(float)[1177:-2,1199:2001]
RAV = np.memmap(prnt+'RAS_'+str(NX)+'x'+str(NY), dtype = '>f4',shape=(NY,NX), mode='r').astype(float)[1177:-2,1199:2001]


## get the file list, I am assuming you have already extracted similar files for your study area
Wf = sorted(glob.glob(prnt+'W_'+str(NX)+'x'+str(NY)+'x'+str(NZ)+'.'+suf+'*'))
Tf = sorted(glob.glob(prnt+'Theta_'+str(NX)+'x'+str(NY)+'x'+str(NZ)+'.'+suf+'*'))
Sf = sorted(glob.glob(prnt+'Salt_'+str(NX)+'x'+str(NY)+'x'+str(NZ)+'.'+suf+'*'))
Uf = sorted(glob.glob(prnt+'U_'+str(NX)+'x'+str(NY)+'x'+str(NZ)+'.'+suf+'*'))
Vf = sorted(glob.glob(prnt+'V_'+str(NX)+'x'+str(NY)+'x'+str(NZ)+'.'+suf+'*'))
Ef = sorted(glob.glob(prnt+'Eta_'+str(NX)+'x'+str(NY)+'.'+suf+'*'))

## load the outputs (here the first file in the folder)
layer_max = 50
w_tot = np.memmap(Wf[0], dtype = '>f4',shape=(NZ,NY,NX), mode='r')[:layer_max,1177:-2,1199:2001]
s_tot = np.memmap(Sf[0], dtype = '>f4',shape=(NZ,NY,NX), mode='r')[:layer_max,1177:-2,1199:2001]
t_tot = np.memmap(Tf[0], dtype = '>f4',shape=(NZ,NY,NX), mode='r')[:layer_max,1177:-2,1199:2001]
u_tot = np.memmap(Uf[0], dtype = '>f4',shape=(NZ,NY,NX), mode='r')[:layer_max,1177:-2,1199:2001]
v_tot = np.memmap(Vf[0], dtype = '>f4',shape=(NZ,NY,NX), mode='r')[:layer_max,1177:-2,1199:2001]
eta_tot = np.memmap(Ef[0], dtype = '>f4',shape=(NY,NX), mode='r')[1177:-2,1199:2001]

## select a layer/depth
layer = 14

z = io.loadmat(prnt+'thk90.mat') # I am assuming you already have this file for the depth/level thickness - let me know if not, or ask Hector or Dimitris for where to find it
depth = z['dpt90'][0,:] # level depth (m)

D = depth[2:layer_max]-depth[:layer_max-2]
D = D[:,None,None]
D = np.tile(D,(1,w_tot.shape[1],w_tot.shape[2]))

## compute finite difference, the super basic way
uz = (u_tot[:-2,:,:]-u_tot[2:,:,:])/D
vz = (v_tot[:-2,:,:]-v_tot[2:,:,:])/D
uz_c = (uz[:,:,:-1]+uz[:,:,1:])/2 # if I remember correctly "_c" means at the center of the cell
vz_c = (vz[:,:-1,:]+vz[:,1:,:])/2
u_z = uz_c[:,1:-1,1:]
v_z = vz_c[:,1:,1:-1]

## load a buoyancy file (I computed it from temperature, salinity, depth and saved it, I think it saved me time instead of recomputing it every time). If you have trouble computing buoyancy, I put a small code that I used below in comments
b = np.load('/Users/siegelma/Documents/Kerguelen_analysis/raw_data/'+suf+'_b.npy')

# ##compute b at a given layer ##
# ref = 1027.5
# rho = dens.densjmd95(s_tot[layer,:,:],t_tot[layer,:,:],depth[layer]) # you can ask Hector the function dens.densjmd95 or use one that you like (e.g., from gsw)
# b[layer,:,:] = 9.81*(1-rho/ref)


b_z = (b[:-2,:,:]-b[2:,:,:])/D
bz = b_z[:,1:-1,1:-1]
b0 = np.zeros((bz.shape[1],bz.shape[2]7))
b0 = b0[None,:]
bz = np.vstack((b0,bz))

# db/dx
b2x = (b[:,:,2:] * DYF[:,2:] - b[:,:,:-2] * DYF[:,:-2])/(RAU[:,1:-1]+RAU[:,:-2])
# db/dx
b2y = (b[:,2:,:] * DXF[2:,:] - b[:,:-2,:] * DXF[:-2,:])/(RAV[1:-1,:]+RAV[:-2,:])
### b_x and b_y are at the cell center ##
b_x = b2x[:,1:-1,:]
b_y = b2y[:,:,1:-1]

Gradb = np.sqrt(b_x**2 + b_y**2)

## Richardson number assuming thermal wind
yc = YC[1:-1,1:-1]
f = gsw.f(yc)
Ri = ((f**2)*bz)/(Gradb[:-1,:,:]**2)

## Richardson number not assuming thermal wind
Ri_real = bz[:-1,:,:]/(u_z**2+v_z**2)


In [3]:
getZlayerThickness(5)

1.93

In [23]:
getZlayerThickness(np.array(range(0,4)))

array([1.  , 1.14, 1.3 , 1.49])

In [7]:
thk = scipy.io.loadmat('../Hector_Python_Scripts/thk90')
thickness = thk['thk90'][0,0:90]

In [21]:
np.array(range(1,4))

array([1, 2, 3])

In [ ]:
diros='/nobackupp11/dmenemen/DYAMOND/c1440_llc2160/mit_output/Salt'
dss = 